In [2]:
import openpyxl
import os
import pandas as pd
import re, datetime
from fuzzywuzzy import fuzz
import numpy as np


/Users/shuyitan/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# os.chdir('/Users/shuyitan')
#restaurant_name = "Qdoba"
#directory_path = "~/Dropbox/Restaurant Menu's shared workspace/Data/Panel Matching/Shuyi's test folder/" + restaurant_name + "_panel.xlsx"
directory_path = "/Users/shuyitan/Dropbox/Restaurant Menu's shared workspace/Data/Panel Matching/Shuyi's test folder/steak_n_shake_panel.xlsx"
data = pd.read_excel(directory_path)
date_list = data.file_date.unique()

data[['Item Type 1','Item Type 2']] = data[['Item Type 1','Item Type 2']].replace(np.nan, '', regex=True)
data['Item_Types'] = data["Item Type 1"] +" "+ data["Item Type 2"]
data["name_and_type"] = data["Item_Name"] +" "+ data["Item Type 1"] +" "+ data["Item Type 2"]

In [ ]:
i = 0
while i <= len(date_list): 
    # starting from the first date t
    current_file = data[data.file_date == date_list[i]]
    # for those who don't have a match in t + 1
    current_file_no_match = current_file[current_file.post_fuzzy == 0]
    if current_file_no_match.shape[0] != 0: 
        for k in current_file_no_match.index: 
            item = current_file_no_match['name_and_type'][k]
            # # starting iterating over data strating from t + 2
            remain_data = data[(data.file_date != date_list[i]) & (data.file_date != date_list[i+1])]
            for j in range(remain_data.index[0], remain_data.index[-1]): 
                item_be_compared = remain_data['name_and_type'][j]
                # calculate similarity
                ratio = fuzz.token_sort_ratio(item,item_be_compared) 
                # if similarity is greater than 85%, they are considered to be fuzzy-matched reoccurren
                if ratio >= 85: 
                    
                    # find items that have a same id as the matched in 
                    matched_item_id = remain_data['id'][j]
                    # replace their ids with the id of item at time t 
                    loc_filter = data['id'] == matched_item_id
                    # once find a match, break the loop
                    data.loc[loc_filter, ['id']] = current_file_no_match['id'][k]
                    break
                
    i += 1              
